In [36]:
!pip install torch torchquantum

  Using cached torch-1.13.0-cp37-cp37m-manylinux1_x86_64.whl (890.2 MB)
  Using cached torchquantum-0.1.4-py3-none-any.whl (119 kB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached torchvision-0.14.0-cp37-cp37m-manylinux1_x86_64.whl (24.3 MB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached qiskit-0.32.1-py3-none-any.whl
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached qiskit_ignis-0.6.0-py3-none-

In [2]:
!pip install pennylane

You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


In [62]:
from importlib import reload

import datasets
import create_gate_circs_np
import train_circ_np

reload(datasets)
reload(create_gate_circs_np)
reload(train_circ_np)

<module 'train_circ_np' from '/root/train_circ_np.py'>

In [ ]:
from create_noise_models import noisy_dev_from_backend
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ, get_circ_params
from train_circ import train_qnn, mse_loss

import numpy as np
import os

dataset = 'fmnist_4'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'angle', 1)

num_qubits = 4
num_embeds = 16
num_params = 18

device_name = 'ibmq_lima'

# dev = qml.device('lightning.qubit', wires=num_qubits)
dev = noisy_dev_from_backend(device_name, num_qubits)

for i in range(600, 800):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                    weights_bounds, [0], 'exp')
    
    
    noiseless_losses = np.genfromtxt(curr_dir + '/val_losses.txt')

    losses_list = []
    accs_list = []
    
    curr_dev_dir = curr_dir + '/' + device_name

#     if not os.path.exists(curr_dev_dir + '/accs_inference_only.txt'):
    if True:
        if not os.path.exists(curr_dev_dir):
            os.mkdir(curr_dev_dir)

        for j in range(5):
            curr_train_dir = curr_dir + '/run_{}'.format(j + 1)
            curr_params = np.genfromtxt(curr_train_dir + '/params_{}.txt'.format(j + 1))[-1]

            val_exps = [circ(x_test[i], curr_params) for i in range(len(x_test))]
            val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()

            acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
#             acc = np.mean(val_loss < 1)

            losses_list.append(val_loss)
            accs_list.append(acc)

        print(np.mean(noiseless_losses), np.mean(losses_list), i + 1)

        np.save(curr_dev_dir + '/val_losses_inference_only.npy', losses_list)
        np.savetxt(curr_dev_dir + '/accs_inference_only.txt', accs_list)
    else:
        print(i)

ibmqfactory.load_account:WARNING:2022-10-24 04:07:02,908: Credentials are already in use. The existing account in the session will be replaced.


1.097606405579725 1.4509932606764828 601
1.2526336407017855 1.3719399828246002 602
1.180082448479918 1.3649659509576535 603
1.0618923402256006 1.3308654578607826 604


In [ ]:
from create_noise_models import get_real_backend_dev, noisy_dev_from_backend
from create_gate_circs import create_batched_gate_circ, get_circ_params
from datasets_nt import load_dataset
from metrics import compute_noise_metric

dataset = 'vowel_2'

x_train, y_train, _, __ = load_dataset(dataset, 'angle', 1)

curr_dir = './experiment_data/{}/trained_circuits'.format(dataset)

num_qubits = 4
num_trial_params = 128
meas_qubits = [0, 1, 2, 3]

num_cdcs = 128
num_shots = 1024

device_name = 'ibmq_belem'

dev = noisy_dev_from_backend(device_name, num_qubits)
noiseless_dev = qml.device('lightning.qubit', wires=num_qubits)

score_tvds = []
actual_tvds = []

for i in range(1000):
    circ_dir = curr_dir + '/circ_{}'.format(i + 1)
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    noisy_circ = create_batched_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 

    noiseless_circ = create_batched_gate_circ(noiseless_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'probs') 
    
    noise_metric_dir = circ_dir + '/noise_metric'
    
    if not os.path.exists(noise_metric_dir):
        os.mkdir(noise_metric_dir)
    
    device_noise_metric_dir = noise_metric_dir + '/{}'.format(device_name)

    if not os.path.exists(device_noise_metric_dir):
        os.mkdir(device_noise_metric_dir)
    
    params = np.random.sample((num_trial_params, weights_bounds[-1])) * 2 * np.pi
    batch_data = x_train[np.random.choice(len(x_train), num_trial_params, False)]
    
    noiseless_res_raw = np.array(noiseless_circ(batch_data, params, shots=num_shots))
    noisy_res_raw = np.array(noisy_circ(batch_data, params, shots=num_shots))

    actual_tvd = 1 - np.mean(np.sum(0.5 * np.abs(noiseless_res_raw - noisy_res_raw), 1))
    
    tvd = compute_noise_metric(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, noiseless_dev, dev, num_cdcs=num_cdcs, num_shots=num_shots)
    
    np.savetxt(device_noise_metric_dir + '/metric_tvd_score.txt', [1 - tvd])
    np.savetxt(device_noise_metric_dir + '/actual_tvd_score.txt', [actual_tvd])
    
    score_tvds.append(1 - tvd)
    actual_tvds.append(actual_tvd)
    
    print(1 - tvd, actual_tvd, i)

# np.savetxt(curr_dir + '/noise_scores.txt', score_tvds)
# np.savetxt(curr_dir + '/noisy_fids.txt', actual_tvds)

ibmqfactory.load_account:WARNING:2022-11-05 17:24:59,216: Credentials are already in use. The existing account in the session will be replaced.
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (94238.37625242842 > 2 * 3734.592789848748). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (94238.37625242842 > 2 * 3734.592789848748). Truncating to maximum value.
  " Truncating to maximum value.", UserWarning)
/usr/local/lib/python3.7/site-packages/qiskit_aer/noise/device/models.py:367: UserWarning: Device model returned an invalid T_2 relaxation time greater than the theoretical maximum value 2 * T_1 (94238.37625242842 > 2 * 37

0.8943939208984375 0.9108200073242188 0
0.9081878662109375 0.91632080078125 1
0.9145660400390625 0.9234237670898438 2
0.855621337890625 0.8656539916992188 3
0.8965072631835938 0.8970489501953125 4
0.8485794067382812 0.8633880615234375 5
0.8626632690429688 0.8824310302734375 6
0.8270797729492188 0.8687515258789062 7
0.8469924926757812 0.8785781860351562 8
0.8934555053710938 0.90130615234375 9
0.8481597900390625 0.8551254272460938 10
0.7269058227539062 0.7571945190429688 11
0.8270721435546875 0.8769683837890625 12


## supernet

In [6]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

import pennylane as qml
import numpy as np
import os

dataset = 'mnist_2'
main_dir = './supernet/mnist_2/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4
meas_qubits = [0]
num_steps = 8000

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [8, 12, 16, 20]
search_nums = [100]
num_trials = [25]

device_name = 'ibm_nairobi'
noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    circ_accs = np.genfromtxt(curr_params_dir + '/{}/searched_circ_accs.txt'.format(device_name))
    circ_losses = np.genfromtxt(curr_params_dir + '/{}/searched_circ_losses.txt'.format(device_name))
    circ_layers = np.genfromtxt(curr_params_dir + '/{}/searched_circ_layers.txt'.format(device_name)).astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/{}/search_{}'.format(device_name, search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
            noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
                noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                noisy_val_loss = np.array([mse_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

#                 acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
#                 noisy_acc = np.mean(np.sum(np.multiply(noisy_val_exps, y_test) > 0, 1) == 2)
                acc = np.mean(val_loss < 1)
                noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
                noisy_losses_list.append(noisy_val_loss)
                noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
            np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
            np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

ibmqfactory.load_account:WARNING:2022-10-13 15:28:25,845: Credentials are already in use. The existing account in the session will be replaced.


0.5 1.0292612075805665 [259320  70916]
[2022-10-13 15:28:28.571 tensorflow-2-3-cpu-p-ml-c5-4xlarge-d57452a0615e3c39f303a80a4c0c:24 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-10-13 15:28:28.608 tensorflow-2-3-cpu-p-ml-c5-4xlarge-d57452a0615e3c39f303a80a4c0c:24 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Step 1 | Sliding Loss Window : 1.397855832389007
Step 51 | Sliding Loss Window : 1.0212690295687283
Step 101 | Sliding Loss Window : 1.1500730289791028
Step 151 | Sliding Loss Window : 1.0077736550065728
Step 201 | Sliding Loss Window : 1.0119776279867287
Step 251 | Sliding Loss Window : 1.0295576518756164
Step 301 | Sliding Loss Window : 1.009455352506303
Step 351 | Sliding Loss Window : 1.027050546647629
Step 401 | Sliding Loss Window : 0.9952755032371955
Step 451 | Sliding Loss Window : 1.0413712863529658
Step 501 | Sliding Loss Window : 1.025507630035909
Step 551 | Sliding Loss Window : 

## train rando circuits

In [ ]:
from datasets import TorchDataset
from datasets_nt import load_dataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss
from create_noise_models import noisy_dev_from_backend

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'mnist_4'
curr_dir = f'./random/{dataset}/'

num_qubits = 4
num_embeds = 16

num_train_steps = 2500
num_test_data = 2000

num_meas_qubits = 2

param_nums = [24, 28, 32, 36, 40]

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

_, _, x_test, y_test = load_dataset(dataset, 'angle', 1)

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=True)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for param_num in param_nums:
    curr_param_dir = curr_dir + f'{param_num}_params/'
    
    for i in range(25):    
        circ_dir = curr_param_dir + f'circ_{i + 1}'

        circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)
        
        losses_list = []
        accs_list = []

        for j in range(5):
            curr_train_dir = circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)

            model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
            opt = torch.optim.SGD(model.parameters(), lr=0.05)

            curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)

            print(curr_loss, curr_acc)

            losses_list.append(curr_loss)
            accs_list.append(curr_acc)
            
            torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        np.savetxt(circ_dir + '/val_losses.txt', losses_list)
        np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 2.8476617336273193
Step 101 | Loss: 1.7598271369934082
Step 201 | Loss: 1.7594096660614014
Step 301 | Loss: 1.8773781061172485
Step 401 | Loss: 1.8681859970092773
Step 501 | Loss: 1.679863452911377
Step 601 | Loss: 1.6744399070739746
Step 701 | Loss: 1.8276506662368774
Step 801 | Loss: 1.7163264751434326
Step 901 | Loss: 1.6526825428009033
Step 1001 | Loss: 1.8605990409851074
Step 1101 | Loss: 1.853203535079956
Step 1201 | Loss: 1.6710652112960815
Step 1301 | Loss: 1.827438473701477
Step 1401 | Loss: 1.8229390382766724
Step 1501 | Loss: 1.8562507629394531
Step 1601 | Loss: 1.698634386062622
Step 1701 | Loss: 1.6277490854263306
Step 1801 | Loss: 1.6898982524871826
Step 1901 | Loss: 1.6240745782852173
Step 2001 | Loss: 1.5250216722488403
Step 2101 | Loss: 1.6312403678894043
Step 2201 | Loss: 1.9689826965332031
Step 2301 | Loss: 1.655059576034546
Step 2401 | Loss: 1.5859121084213257
1.7677753773746174 0.3305
Step 1 | Loss: 2.9976119995117188
Step 101 | Loss: 1.9172185659408